#### Project: Portfolio Optimization <br> Author: Dhruv Singh <br> Date Updated: 8/2/2023

# Portfolio Optimization

In [1]:
# libraries
import math
#import eikon as ek
import numpy as np  
import pandas as pd
import cufflinks as cf  
import scipy.optimize as sco
#from dotenv import load_dotenv
from IPython.display import display, Markdown
import warnings
import os

cf.set_config_file(offline=True)  
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

### Part 1: EOD Closing Price Data

In [2]:
'''
load_dotenv("eikon.env")
eikon_api_key = os.getenv("eikon_api_key")
ek.set_app_key(eikon_api_key)
'''

'\nload_dotenv("eikon.env")\neikon_api_key = os.getenv("eikon_api_key")\nek.set_app_key(eikon_api_key)\n'

In [3]:
# rics = ['BRKa', 'US10YT=RR', 'US1MT=RR', 'AMT', 'GLD', 'BLK', 'BX', 'IVZ', 'LAZ', 'KKR', 'IEP.O']
rics = ['BRKa', # stock: berkshire
        'US10YT=RR', # bond: 10 year
        'AMT', # reit: american towers
        'GLD', # commodity: gold
        'BLK', # hedge fund: blackrock
        'BX', # pe: blackstone
        'IVZ', # hedge fund (etfs): invesco
        'LAZ', # hedge fund: lazard
        'KKR', # hedge fund: kkr
        'IEP.O'] # hedge fund: icahn enterprises

'''
start_date = '2011-01-01'
end_date = '2024-06-01'

# empty df
data = pd.DataFrame()

# looping 
for year in range(2011, 2024):
    start_date_year = f'{year}-01-01'
    end_date_year = f'{year}-06-01'
    df = ek.get_timeseries(rics, fields='CLOSE', start_date=start_date_year, end_date=end_date_year)
    
    # concatenating
    data = pd.concat([data, df])

data.to_csv('0_readonly/portfolio_data.csv')
'''

"\nstart_date = '2011-01-01'\nend_date = '2024-06-01'\n\n# empty df\ndata = pd.DataFrame()\n\n# looping \nfor year in range(2011, 2024):\n    start_date_year = f'{year}-01-01'\n    end_date_year = f'{year}-06-01'\n    df = ek.get_timeseries(rics, fields='CLOSE', start_date=start_date_year, end_date=end_date_year)\n    \n    # concatenating\n    data = pd.concat([data, df])\n\ndata.to_csv('0_readonly/portfolio_data.csv')\n"

#### Data: Portfolio (2011-2023)
* 6 hedge funds / p.e.
* 1 stock
* 1 bond
* 1 commodity
* 1 reit

In [4]:
# reading in data
data = pd.read_csv('0_readonly/portfolio_data.csv', index_col='Date', parse_dates=['Date']) 
data.head()

,BRKa,US10YT=RR,AMT,GLD,BLK,BX,IVZ,LAZ,KKR,IEP.O
Date,,,,,,,,,,
2011-01-03,120498.0,3.336,51.63,138.00,190.19,14.503767,24.46,35.366027,14.50,34.294210
2011-01-04,120200.0,3.338,51.47,134.75,190.04,14.680284,24.41,35.143150,14.45,34.275984
2011-01-05,121300.0,3.463,50.76,134.37,192.00,14.670477,24.39,36.346683,15.14,34.534030
2011-01-06,120600.0,3.403,50.62,133.83,189.93,14.690090,24.35,37.050972,15.23,34.514844
2011-01-07,119681.0,3.326,50.50,133.58,188.36,14.621445,24.33,36.587389,14.97,35.243896


In [5]:
data.tail()

,BRKa,US10YT=RR,AMT,GLD,BLK,BX,IVZ,LAZ,KKR,IEP.O
Date,,,,,,,,,,
2023-05-25,484000.00,3.815,182.56,180.20,660.52,83.53,14.70,28.15,50.70,20.63
2023-05-26,486650.00,3.820,182.18,180.92,672.30,85.70,14.89,28.76,51.68,20.65
2023-05-30,489224.73,3.696,182.00,182.04,673.58,86.40,14.82,28.93,51.67,22.38
2023-05-31,488023.98,3.637,184.44,182.32,657.55,85.64,14.38,28.69,51.49,22.57
2023-06-01,492000.01,3.608,187.01,183.76,668.84,87.14,14.74,29.10,52.43,21.81


In [6]:
# dropping missing values
data.dropna(inplace=True) 
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1351 entries, 2011-01-03 to 2023-06-01
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   BRKa       1351 non-null   float64
 1   US10YT=RR  1351 non-null   float64
 2   AMT        1351 non-null   float64
 3   GLD        1351 non-null   float64
 4   BLK        1351 non-null   float64
 5   BX         1351 non-null   float64
 6   IVZ        1351 non-null   float64
 7   LAZ        1351 non-null   float64
 8   KKR        1351 non-null   float64
 9   IEP.O      1351 non-null   float64
dtypes: float64(10)
memory usage: 116.1 KB


### Preparing Data for Analysis

#### Returns

In [7]:
# plotting normalized closing price 
data.normalize().iplot(kind='lines')

In [8]:
# logarithmic returns
rets = np.log(data / data.shift(1))
rets.head()

,BRKa,US10YT=RR,AMT,GLD,BLK,BX,IVZ,LAZ,KKR,IEP.O
Date,,,,,,,,,,
2011-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-01-04,-0.002476,0.000599,-0.003104,-0.023832,-0.000789,0.012097,-0.002046,-0.006322,-0.003454,-0.000532
2011-01-05,0.009110,0.036763,-0.013890,-0.002824,0.010261,-0.000668,-0.000820,0.033673,0.046646,0.007500
2011-01-06,-0.005788,-0.017478,-0.002762,-0.004027,-0.010840,0.001336,-0.001641,0.019192,0.005927,-0.000556
2011-01-07,-0.007649,-0.022887,-0.002373,-0.001870,-0.008301,-0.004684,-0.000822,-0.012591,-0.017219,0.020903


In [9]:
rets.tail()

,BRKa,US10YT=RR,AMT,GLD,BLK,BX,IVZ,LAZ,KKR,IEP.O
Date,,,,,,,,,,
2023-05-25,-0.005244,0.025486,-0.004536,-0.009665,0.008392,0.004439,-0.005427,-0.011655,0.025166,-0.148804
2023-05-26,0.005460,0.001310,-0.002084,0.003988,0.017677,0.025647,0.012842,0.021438,0.019145,0.000969
2023-05-30,0.005277,-0.032999,-0.000989,0.006171,0.001902,0.008135,-0.004712,0.005894,-0.000194,0.080452
2023-05-31,-0.002457,-0.016092,0.013318,0.001537,-0.024086,-0.008835,-0.030139,-0.008330,-0.003490,0.008454
2023-06-01,0.008114,-0.008006,0.013838,0.007867,0.017024,0.017364,0.024727,0.014190,0.018091,-0.034253


In [10]:
# visualizing distribution of log returns
rets.iplot(kind='histogram', subplots=True)

In [11]:
# daily mean returns
rets.mean()

BRKa         0.001042
US10YT=RR    0.000058
AMT          0.000953
GLD          0.000212
BLK          0.000931
BX           0.001328
IVZ         -0.000375
LAZ         -0.000144
KKR          0.000952
IEP.O       -0.000335
dtype: float64

In [12]:
# annualized mean returns
rets.mean() * 252

BRKa         0.262611
US10YT=RR    0.014631
AMT          0.240251
GLD          0.053457
BLK          0.234737
BX           0.334713
IVZ         -0.094542
LAZ         -0.036403
KKR          0.239928
IEP.O       -0.084487
dtype: float64

In [13]:
# mean annual returns
(rets.mean() * 252).iplot(kind='bar')

#### Volatility (Risk Measure)

In [14]:
# daily volatilities
rets.std()

BRKa         0.016959
US10YT=RR    0.040567
AMT          0.020039
GLD          0.012327
BLK          0.025460
BX           0.034240
IVZ          0.036300
LAZ          0.032247
KKR          0.032291
IEP.O        0.031954
dtype: float64

In [15]:
# annualized volatilities
rets.std() * math.sqrt(252)  

BRKa         0.269208
US10YT=RR    0.643975
AMT          0.318116
GLD          0.195687
BLK          0.404169
BX           0.543540
IVZ          0.576249
LAZ          0.511903
KKR          0.512610
IEP.O        0.507247
dtype: float64

In [16]:
# annualized volatilities
(rets.std() * math.sqrt(252)).iplot(kind='bar')

In [17]:
# annualized covariance matrix by column
data.cov() * 252

,BRKa,US10YT=RR,AMT,GLD,BLK,BX,IVZ,LAZ,KKR,IEP.O
BRKa,3.551380e+12,2.478288e+06,1.835329e+09,3.640351e+08,5.439244e+09,8.307386e+08,-9.638335e+07,5.285533e+07,4.029802e+08,-6.619506e+07
US10YT=RR,2.478288e+06,1.326483e+02,-2.692110e+03,-1.601114e+02,-1.262355e+03,1.041108e+02,1.380936e+02,3.240966e+02,2.487087e+02,-2.364812e+02
AMT,1.835329e+09,-2.692110e+03,1.188057e+06,2.083185e+05,2.980495e+06,4.363357e+05,-7.913689e+04,1.484979e+04,2.064900e+05,-3.951016e+04
GLD,3.640351e+08,-1.601114e+02,2.083185e+05,1.386712e+05,5.662726e+05,1.125379e+05,-2.866362e+04,-1.945299e+04,6.122402e+04,-5.389729e+04
BLK,5.439244e+09,-1.262355e+03,2.980495e+06,5.662726e+05,9.310935e+06,1.298647e+06,-1.218853e+05,1.367052e+05,6.512557e+05,-6.859586e+04
BX,8.307386e+08,1.041108e+02,4.363357e+05,1.125379e+05,1.298647e+06,2.270204e+05,-2.283422e+04,6.537543e+03,1.099807e+05,-1.401461e+04
IVZ,-9.638335e+07,1.380936e+02,-7.913689e+04,-2.866362e+04,-1.218853e+05,-2.283422e+04,1.473615e+04,8.639787e+03,-1.062953e+04,2.046012e+04
LAZ,5.285533e+07,3.240966e+02,1.484979e+04,-1.945299e+04,1.367052e+05,6.537543e+03,8.639787e+03,1.670394e+04,4.013260e+03,1.567435e+04
KKR,4.029802e+08,2.487087e+02,2.064900e+05,6.122402e+04,6.512557e+05,1.099807e+05,-1.062953e+04,4.013260e+03,5.712761e+04,-5.575090e+03
IEP.O,-6.619506e+07,-2.364812e+02,-3.951016e+04,-5.389729e+04,-6.859586e+04,-1.401461e+04,2.046012e+04,1.567435e+04,-5.575090e+03,9.016535e+04


### Portfolio Optimization: Equal Weights

In [18]:
def portfolio_return(symbols, weights):
    return np.dot(rets[symbols].mean() * 252, weights)

In [19]:
def portfolio_volatility(symbols, weights):
    return math.sqrt(np.dot(weights, np.dot(rets[symbols].cov() * 252, weights)))

In [20]:
# equal weights
weights = len(rics) * [1 / len(rics)]
weights

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [21]:
# portfolio returns using equal weights
portfolio_return(rics, weights)

0.11648982347633945

In [22]:
# volatility using covariance matrix
np.dot(weights, np.dot(rets.cov() * 252, weights))

0.09894103441818411

In [23]:
# portfolio volatility using equal weights
portfolio_volatility(rics, weights)

0.31454893803378847

### Portfolio Statistics: Two Instruments

In [24]:
# case: two instrument
fis = ['BRKa', 'BX']

In [25]:
# simulating compositions
w = np.random.random((500, len(fis))) 

In [26]:
# normalizing to add to 1
w = (w.T / w.sum(axis=1)).T 

In [27]:
# calculating volatility-return for simulated weights
mvp = [(portfolio_volatility(fis, weights),
       portfolio_return(fis, weights))
         for weights in w]

In [28]:
# converting to dataframe
mvp = pd.DataFrame(np.array(mvp), columns=['volatility', 'return'])
mvp.iloc[:5]

,volatility,return
0,0.328962,0.290726
1,0.332129,0.291573
2,0.439449,0.315313
3,0.487727,0.324537
4,0.368714,0.300493


In [29]:
# visualizing volatility-return for different weights
mvp.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

### Portfolio Statistics: All Instruments

In [30]:
# simulating compositions
w = np.random.random((2500, len(rics))) 

In [31]:
# normalizing to add to 1
w = (w.T / w.sum(axis=1)).T 

In [32]:
# calculating volatility-return for simulated weights
mvp = [(portfolio_volatility(rics, weights),
       portfolio_return(rics, weights))
         for weights in w]

In [33]:
# converting to dataframe
mvp = pd.DataFrame(np.array(mvp), columns=['volatility', 'return'])
mvp.iloc[:5]

,volatility,return
0,0.307183,0.048503
1,0.317037,0.102791
2,0.316794,0.101094
3,0.309742,0.116534
4,0.328604,0.160787


In [34]:
# visualizing volatility-return for different weights
mvp.iplot(x='volatility', y='return', kind='scatter', mode='markers', color='red')

### Portfolio Optimization: Minimizing Volatility

In [35]:
# bounds for asset weights
bounds = len(rics) * [(0, 1)]
bounds

[(0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1)]

In [36]:
# ensuring weights sum to 1
constraints = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1} 

In [37]:
# minimizing portfolio volatility
res = sco.minimize(lambda x: portfolio_volatility(rics, x),
                   len(rics) * [1 / len(rics)],
                   bounds=bounds,
                   constraints=constraints
                  )

In [38]:
# results
res

     fun: 0.15138806688651948
     jac: array([0.15191929, 0.15305229, 0.15156109, 0.15102613, 0.20106824,
       0.22623222, 0.23814757, 0.2262693 , 0.24008676, 0.15241497])
 message: 'Optimization terminated successfully'
    nfev: 89
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([8.66831877e-02, 6.75865032e-02, 1.77607196e-01, 6.30795244e-01,
       7.45388994e-17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       4.53196508e-17, 3.73278690e-02])

In [39]:
# minimum volatility
res['fun']

0.15138806688651948

In [40]:
# optimal portfolio composition
for r in zip(rics, res['x']):
    print('%7s | %7.3f' % (r[0], r[1]))

   BRKa |   0.087
US10YT=RR |   0.068
    AMT |   0.178
    GLD |   0.631
    BLK |   0.000
     BX |   0.000
    IVZ |   0.000
    LAZ |   0.000
    KKR |   0.000
  IEP.O |   0.037
